In [1]:
import numpy as np
import os
import pandas as pd
import pickle as pk
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.backend import categorical_crossentropy
from tensorflow.keras.preprocessing import sequence
from nltk.translate.bleu_score import sentence_bleu

In [2]:
# Conversational Model Metric
def perplexity(y_true, y_pred):
    cross_entropy = categorical_crossentropy(y_pred, y_true)
    return pow(2, cross_entropy)

In [3]:
os.chdir('../../logs/')

In [4]:
# Especify model directory
modelDir = 'logs_trained_models_server/model_epochs-155_batch-32_hidden-128_embed-264_v0'



In [5]:
# Load variables
word2index_inp = pk.load(open(os.path.join(modelDir,'word2index_inp.pk'), 'rb'))
index2word_inp = pk.load(open(os.path.join(modelDir,'index2word_inp.pk'),'rb'))

word2index_out = pk.load(open(os.path.join(modelDir,'word2index_out.pk'), 'rb'))
index2word_out = pk.load(open(os.path.join(modelDir,'index2word_out.pk'),'rb'))

X_input = pk.load(open(os.path.join(modelDir,'x_inp.pk'),'rb'))
y_input = pk.load(open(os.path.join(modelDir,'y_inp.pk'),'rb'))


In [6]:
# Load models
model = load_model(os.path.join(modelDir, 'chpts/35_2.43.chpt'), custom_objects={'perplexity': perplexity})
encoder_model = load_model(os.path.join(modelDir, 'encoder_model_128_32_155_264.h5'))
decoder_model = load_model(os.path.join(modelDir, 'decoder_model_128_32_155_264.h5'))

/Users/cortega/Documents/TF_chatbot/chatbot_env/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [7]:
encoder_model.compile(optimizer=Nadam(), loss=perplexity)

In [8]:
decoder_model.compile(optimizer=Nadam(), loss=perplexity)

In [9]:
def decode_sequence(input_seq, target_len, v=0, counter=None, n=None):
    '''Generates answer from given sequence of word indexes'''
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, target_len))
    # Populate the first token of target sequence with the start token.
    target_seq[0, 0] = word2index_out['GO']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decode_sentence = []
    prediction_index = 0
    while not stop_condition:
        ## Print sequence for debugging
        #print(target_seq)
        output_word_index, h, c = decoder_model.predict([target_seq] + states_value)
        #print(output_word_index.shape)

        # Sample a token
        sampled_word_index = np.argmax(output_word_index[0, -1, :])
        sampled_word = index2word_out[sampled_word_index]
        decode_sentence.append(sampled_word)

        # Exit condition: either hit max length
        # or find stop token.
        if (sampled_word == 'EOS' or len(decode_sentence) >= target_len):
            stop_condition = True
        else:
            # Update the target sequence (of length 1).
            target_seq[0, prediction_index+1] = sampled_word_index
            prediction_index += 1

    ## Print sentece for debugging
    # print(' '.join(decode_sentence))

    # Update states
    states_value = [h, c]
    
    if v == 1:
        counter += 1
        print('{} of {}'.format(counter, n), end='\r')

    return ' '.join(decode_sentence)

In [10]:
def indexseq2text(indexes_seq, index2word):
    string_ = []
    for val in indexes_seq:
        string_.append(index2word[val])
    
    return ' '.join(string_)

In [11]:
def text2indexseq(tokens, word2index):
    indexseq = []
    for word in tokens:
        indexseq.append(word2index[word])
    
    return indexseq

In [13]:
indexseq = text2indexseq(['Buenos','dias','solicitar','un','certificado','de','egreso'], word2index_out)
indexseq = np.array(indexseq)


In [14]:
indexseq = sequence.pad_sequences([indexseq], maxlen=200, padding='post')

In [15]:
%%time
decode_sequence(input_seq=indexseq,
                target_len=200)

CPU times: user 41.8 s, sys: 17.5 s, total: 59.3 s
Wall time: 16.1 s


'listado listado listado listado puedes puedes puedes pedido puede puede puede puede puede puede puede puede puede puede puede puede puede puede puede puede puede puede puede puede hasta hasta hasta menos menos menos julio julio julio disposicion Requisitos vez ese ese les perteneces les correcto correcto 08 08 podran podran Luego Luego Luego Luego Luego Luego su su su su su blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco blanco

In [ ]:
result_df = pd.DataFrame()
result_df['input_string']  = [indexseq2text(X_input[i], index2word_inp) for i in range(X_input.shape[0])]
result_df['real_string'] = [indexseq2text(y_input[i], index2word_out) for i in range(y_input.shape[0])]

In [ ]:
result_df['predictions_string'] = [decode_sequence(X_input[i:i+1], y_input.shape[1], 1, i, y_input.shape[0]) for i in range(X_input.shape[0]-1)]




In [ ]:
np.random.seed(0)
sample = np.random.randint(0, y_input.shape[0]-1, size=10)
X_sample, y_sample = X_input[sample, :], y_input[sample, :]

In [ ]:
with open('test_samples.txt','wb') as f:
    sep = '-'*80
    for i in range(X_input.shape[0]):
        input_string = indexseq2text(X_input[i], index2word_inp)
        print(input_string)
        print(sep)
        real_string = indexseq2text(y_input[i], index2word_out)
        print(real_string)
        if i < X_input.shape[0]-1:
            predicted_string = decode_sequence(X_input[i:i+1], y_input.shape[1])
        print(predicted_string)
        
            
        
        f.write()

result_df = pd.DataFrame()
result_df['input_string']  = [indexseq2text(X_input[i], index2word_inp) for i in range(X_input.shape[0])]
result_df['real_string'] = [indexseq2text(y_input[i], index2word_out) for i in range(y_input.shape[0])]

In [ ]:
result_df['predictions_string'] = [decode_sequence(X_input[i:i+1], y_input.shape[1], 1, i, y_input.shape[0]) for i in range(X_input.shape[0]-1)]




In [ ]:
sep = '-'*80
for i in range(X_sample.shape[0]):
    input_string = indexseq2text(X_sample[i], index2word_inp)
    print(input_string)
    print(sep)
    real_string = indexseq2text(y_sample[i], index2word_out)
    print(real_string)
    if i < X_input.shape[0]-1:
        predicted_string = decode_sequence(X_sample[i:i+1], y_sample.shape[1])
    print(X_sample[i:i+1])
    print(predicted_string)
    print('*'*80)

In [ ]:
decode_sequence(np.array([range(200)]), 200)